# Table of Contents
 <p><div class="lev1"><a href="#Task-1.-Compiling-Ebola-Data"><span class="toc-item-num">Task 1.&nbsp;&nbsp;</span>Compiling Ebola Data</a></div>
 <div class="lev1"><a href="#Task-2.-RNA-Sequences"><span class="toc-item-num">Task 2.&nbsp;&nbsp;</span>RNA Sequences</a></div>
 <div class="lev1"><a href="#Task-3.-Class-War-in-Titanic"><span class="toc-item-num">Task 3.&nbsp;&nbsp;</span>Class War in Titanic</a></div></p>

In [1]:
DATA_FOLDER = 'Data/' # Use the data folder provided in Tutorial 02 - Intro to Pandas.

## Task 1. Compiling Ebola Data

The `DATA_FOLDER/ebola` folder contains summarized reports of Ebola cases from three countries (Guinea, Liberia and Sierra Leone) during the recent outbreak of the disease in West Africa. For each country, there are daily reports that contain various information about the outbreak in several cities in each country.

Use pandas to import these data files into a single `Dataframe`.
Using this `DataFrame`, calculate for *each country*, the *daily average per month* of *new cases* and *deaths*.
Make sure you handle all the different expressions for *new cases* and *deaths* that are used in the reports.

In [211]:
'''
# Write your answer here
the ebola folder contains subfolders for each country, for each country there are a bunch of 
.csv files for each date, need to combine all of these into 1 single dataframe
'''

'Explanation:'
'''Instead of importing all the files into a single dataframe, we will import one at a time
compute the necessary values and add those to a datastructure. We have to pay attention to the fact
that we are finding averages over each month'''

'''
def readFolder(path): 
    allFiles=glob.glob(path) 
    frame=pd.DataFrame()
    l=[] 
    for file in allFiles: 
        df=pd.read_csv(file) 
        l.append(df) 
        frame=pd.concat(l) 
        return frame 
    
guinea = readFolder(DATA_FOLDER+'/ebola/guinea_data/*.csv') 
guinea=guinea.fillna(value=0) print(guinea.head()) 
liberia = readFolder(DATA_FOLDER+'/ebola/liberia_data/*.csv') 
sl = readFolder(DATA_FOLDER+'/ebola/sl_data/*.csv') 
allCities=[guinea,liberia,sl] df = pd.DataFrame() 
df=pd.concat(allCities) 
print(df)'''



'''
test = pd.read_csv(DATA_FOLDER+'ebola/guinea_data/2014-08-04.csv')
print(test[3:4]['Totals'])
print(test[15:16]['Totals'])
'''

'''

'''


'''need to preprocess the data, maybe instead of date just use month since we're dealing with monthly values'''


'''a[a['Description']=='Total new cases registered so far']'''
'''a=total[['Description','Variable','Totals','Date','Country']]

'''


'fixing column names, for example training school spelling mistake in SL'
'date formats are messed uo'
'''
print(guinea.columns)
print(liberia.columns)
print(sl.columns)
'''
'''
print(guinea['Date'].head())

print(liberia['Date'].head())
'''
'''
print(guinea.columns)
print(liberia.columns)
print(sl.columns)
'''
'''make sum column at the end, check for errors'''

'''stri=guinea['Date'][0:1].values[0]
dt = datetime.strptime(stri, '%Y-%m-%d')
file['Date'].values[0]=str(dt.month)'''
'''
for row in guinea:
    print(row)
'''

'''guinea['Date']=1'''
#'print(guinea['Date'].values)''''''
'''
print(guinea.columns)
print(liberia.columns)
print(sl.columns)
'''


'''liberia.rename(columns={'Variable': 'Description','National':'Totals'}, inplace=True)
sl.rename(columns={'variable': 'Description','National':'Totals','date':'Date'}, inplace=True)
'''
'''
def changeDate(file):
    for row in file:        
  
'''

'have to do rename on total frame'
#Guinea
#Total new cases registered so far

#Liberia
#New Case/s (Suspected)
#New Case/s (Probable)
#New case/s (confirmed)

#SL
#new_confirmed



#total[(total['Country']=='guinea')&(total['Description']=='New deaths registered')][['Description','Country','Date','Totals']]
                    
#grouped = total.groupby(['Date','Description','Country']).mean()
#print(grouped)

#not all date have same format DD/MM/YYYY
#total['Date'] = total['Date'].apply(lambda x:if dateIsValid(print(datetime.strptime(x, '%Y-%m-%d').month))
#total['Date'] = total['Date'].apply(getMonth)
#western area combined = western are rural + urban ??

SyntaxError: EOL while scanning string literal (<ipython-input-211-11bb352a8350>, line 58)

In [387]:
import glob
import datetime as dt
import pandas as pd

def readFiles(path,name):
    df=pd.DataFrame()
    allFiles=glob.glob(DATA_FOLDER+path)
    for file in allFiles:
        df=df.append(pd.read_csv(file))
    df['Country']=name
    return df


def readAll():
    df=pd.DataFrame()
    df=df.append(readFiles('ebola/guinea_data/*','guinea'))
    df=df.append(readFiles('ebola/liberia_data/*','liberia'))
    df=df.append(readFiles('ebola/sl_data/*','sl'))
    return df

def mergeColumns(df):
    df['Date']=df['Date'].fillna(df['date'])
    df['Description']=df['Description'].fillna(df['Variable'])
    df['Description']=df['Description'].fillna(df['variable'])
    df['Totals']=df['Totals'].fillna(df['National'])
    df['Police training School']=df['Police training School'].fillna(df['Police traning School'])
    return df

def dropColumns(df):
    df=df.drop('date', axis=1)
    df=df.drop('Variable', axis=1)
    df=df.drop('variable', axis=1)
    df=df.drop('National', axis=1)
    df=df.drop('Police traning School',axis=1)
    return df
    
    
def stripDate(date):
    if ('/' in date):
        try :
            return dt.datetime.strptime(date, "%m/%d/%Y").month
        except ValueError:
            return dt.datetime.strptime(date, "%m/%d/%y").month
    return dt.datetime.strptime(date, "%Y-%m-%d").month

def changeDate(df):
    df['Date'] = df['Date'].map(stripDate)
    return df
    
def preprocess(df):
    df=mergeColumns(df)
    df=dropColumns(df)
    df=changeDate(df)
    df=df.fillna(value=0)
    return df

'''
Using this DataFrame, calculate for each country, 
the daily average per month of new cases and deaths. 
Make sure you handle all the different expressions for 
new cases and deaths that are used in the reports.

calc mean over each month (8,9,10,...)
'''

df=readAll()
df=preprocess(df)


#Calculate for guinea
strNewCases='Total new cases registered so far'
guinea=df[(df['Country']=='guinea')&(df['Description']==strNewCases)][['Totals','Date']]
guinea = guinea.astype(float)   
guineaNewCaseMean = guinea.groupby(['Date'])['Totals'].mean()


strNewDeaths='New deaths registered'
strNewDeaths2='New deaths registered today'

guinea=df[(df['Country']=='guinea')&((df['Description']==strNewDeaths)|(df['Description']==strNewDeaths2))][['Totals','Date']]
guinea = guinea.astype(float)   
guineaNewDeathMean = guinea.groupby(['Date'])['Totals'].mean()

print('Daily average new cases per month')
print(guineaNewCaseMean)
print('\n')
print('Daily average new deaths per month')
print(guineaNewDeathMean)


#guinea=df[(df['Country']=='guinea')]
#print(guinea['Description'].unique())

Daily average new cases per month
Date
8.0     25.800
9.0     19.625
10.0    34.000
Name: Totals, dtype: float64


Daily average new deaths per month
Date
8.0      3.4000
9.0      3.5625
10.0    15.0000
Name: Totals, dtype: float64


## Task 2. RNA Sequences

In the `DATA_FOLDER/microbiome` subdirectory, there are 9 spreadsheets of microbiome data that was acquired from high-throughput RNA sequencing procedures, along with a 10<sup>th</sup> file that describes the content of each. 

Use pandas to import the first 9 spreadsheets into a single `DataFrame`.
Then, add the metadata information from the 10<sup>th</sup> spreadsheet as columns in the combined `DataFrame`.
Make sure that the final `DataFrame` has a unique index and all the `NaN` values have been replaced by the tag `unknown`.

In [ ]:
# Write your answer here

## Task 3. Class War in Titanic

Use pandas to import the data file `Data/titanic.xls`. It contains data on all the passengers that travelled on the Titanic.

In [ ]:
from IPython.core.display import HTML
HTML(filename=DATA_FOLDER+'/titanic.html')

For each of the following questions state clearly your assumptions and discuss your findings:
1. Describe the *type* and the *value range* of each attribute. Indicate and transform the attributes that can be `Categorical`. 
2. Plot histograms for the *travel class*, *embarkation port*, *sex* and *age* attributes. For the latter one, use *discrete decade intervals*. 
3. Calculate the proportion of passengers by *cabin floor*. Present your results in a *pie chart*.
4. For each *travel class*, calculate the proportion of the passengers that survived. Present your results in *pie charts*.
5. Calculate the proportion of the passengers that survived by *travel class* and *sex*. Present your results in *a single histogram*.
6. Create 2 equally populated *age categories* and calculate survival proportions by *age category*, *travel class* and *sex*. Present your results in a `DataFrame` with unique index.

In [ ]:
# Write your answer here